In [ ]:
import os
import sys
import time
import numpy as np
import cv2
import json
import glob
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np 
from collections import namedtuple
from pycocotools import mask as mask_utils
from PIL import Image
from scipy.spatial import Delaunay
from tqdm.notebook import tqdm

In [ ]:
from helpers_cal import *
from helpers_masktransfer import *
from helpers_analysis import *

The path below can be adjusted based on different dataset

In [ ]:
# COLMAP Path
colmap_scripts_path = "/userdata/chung-yu.wei/colmap/scripts/python"
if colmap_scripts_path not in sys.path:
    sys.path.append(colmap_scripts_path)

# Core data Paths
base_path = "/userdata/chung-yu.wei/3DRealCar/data/2024_07_02_15_18_44"
sparse_model_path = os.path.join(base_path, "colmap_processed/sparse/0")
rgb_images_path = os.path.join(base_path, "colmap_processed/input")
masks_path = os.path.join(base_path, "colmap_processed/masks/sam")

# Depth map path
depth_maps_path = os.path.join(base_path, "3dscanner_origin")

# json component path
component_json_path = os.path.join(base_path, "2Dto2D_maskmatching/2024_07_02_15_18_44.json")

import read_write_model
cameras, images, points3D = read_write_model.read_model(path=sparse_model_path, ext=".bin")
print("COLMAP model loaded successfully.")

In [ ]:
# --- 3. Create Mappings from Image ID to File Paths ---
image_id_to_rgb_path = {}
image_id_to_depth_path = {}
image_id_to_mask_path = {}

for img_id, img_data in images.items():
    image_name = img_data.name
    base_name, ext = os.path.splitext(image_name)
    number_part = base_name.split('_')[-1]

    # Map to RGB image path
    rgb_path = os.path.join(rgb_images_path, image_name)
    if os.path.exists(rgb_path):
        image_id_to_rgb_path[img_id] = rgb_path
        
    # Map to depth map path
    depth_filename = f"depth_{number_part}.png"
    depth_path = os.path.join(depth_maps_path, depth_filename)
    if os.path.exists(depth_path):
        image_id_to_depth_path[img_id] = depth_path

    # Map to general vehicle mask path
    mask_path = os.path.join(masks_path, image_name)
    if os.path.exists(mask_path):
        image_id_to_mask_path[img_id] = mask_path

print(f"✅ Created path mappings for {len(image_id_to_rgb_path)} images.")

# --- 4. Find All Valid Image IDs ---
# An image is valid if it has an RGB, Mask, and Depth file
valid_image_ids = [
    img_id for img_id, path in image_id_to_rgb_path.items()
    if os.path.exists(path) and
       os.path.exists(image_id_to_mask_path.get(img_id, '')) and
       os.path.exists(image_id_to_depth_path.get(img_id, ''))
]
print(f"✅ Found {len(valid_image_ids)} images with all required files (RGB, Mask, Depth).")

print("\n--- Setup Complete ---")

In [ ]:
# Define the list of components you want to analyze
target_components = [
    "bonnet",
    "bumper_f/cover",
    "windshield_f",
    "headlamp_l_assy",
    "mirror_l_assy",
    "grille",
    "door_fl_assy"
]

In [ ]:
# --- Pre-processing: Indexing and sorting all views for each component ---
# Load component data
image_name_to_anns, cat_id_to_name = load_component_data(component_json_path)

print("--- Pre-processing: Finding the best view for each component ---")

# Create a map: {component_name: [(area, image_id), ...]}
component_area_map = {}

# Create a reverse map to find image_id from normalized name
normalized_name_to_id = {
    os.path.splitext(os.path.basename(img.name))[0]: img_id
    for img_id, img in images.items()
}

# Iterate through all annotated images
for normalized_name, annotations in image_name_to_anns.items():
    if normalized_name in normalized_name_to_id:
        image_id = normalized_name_to_id[normalized_name]
        for ann in annotations:
            component_name = cat_id_to_name[ann['category_id']]
            
            # Decode mask and calculate area
            mask = mask_utils.decode(ann['segmentation'])
            area = np.sum(mask > 0)
            
            # Store the data
            if component_name not in component_area_map:
                component_area_map[component_name] = []
            component_area_map[component_name].append((area, image_id))

# Sort the lists for each component by area (largest first)
for component_name in component_area_map:
    component_area_map[component_name].sort(key=lambda x: x[0], reverse=True)

print("Pre-processing complete. Stored best views for all components.")

print("--- Pre-processing: Indexing and sorting all views for each component ---")
component_to_views_map = {}
for normalized_name, annotations in image_name_to_anns.items():
    if normalized_name in normalized_name_to_id:
        image_id = normalized_name_to_id[normalized_name]
        for ann in annotations:
            component_name = cat_id_to_name[ann['category_id']]
            if component_name not in component_to_views_map:
                component_to_views_map[component_name] = []
            mask = mask_utils.decode(ann['segmentation'])
            area = np.sum(mask > 0)
            component_to_views_map[component_name].append((area, image_id))

# Sort each list by area in descending order
for name in component_to_views_map:
    component_to_views_map[name].sort(key=lambda x: x[0], reverse=True)
print("Pre-processing complete. Indexed and sorted all views")

The Quickest & Best version currently, depth_tolerance with 0.5 is the best for right now

In [ ]:
def transfer_mask_v5(
    source_id, target_id, source_component_mask, component_name,
    images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path,
    subsample_step=9,
    depth_tolerance=0.50,
    kernel_size=11
):
    """
    V5 ("Tuned & Filtered"): A tunable version that also filters out
    source masks that are too small, using a component-specific threshold.
    """
    source_h, source_w, _ = cv2.imread(image_id_to_rgb_path[source_id]).shape
    mask_area = np.sum(source_component_mask > 0)
    image_area = source_h * source_w
    
    # --- Get the component-specific threshold and apply the filter ---
    min_mask_ratio = get_mask_size_threshold(component_name)
    if (mask_area / image_area) < min_mask_ratio:
        print(f"    -> Source mask is too small ({mask_area / image_area:.2%}), below threshold of {min_mask_ratio:.2%}. Skipping this view.")
        return None

    # (The internal logic is the same as your v3_fast, but uses the new tunable parameters)
    target_rgb_for_size = cv2.imread(image_id_to_rgb_path[target_id]); target_h, target_w, _ = target_rgb_for_size.shape
    source_depth_map_raw = cv2.imread(image_id_to_depth_path[source_id], cv2.IMREAD_UNCHANGED)
    inpaint_mask = np.where(source_depth_map_raw == 0, 255, 0).astype(np.uint8)
    source_depth_map = cv2.inpaint(source_depth_map_raw, inpaint_mask, 3, cv2.INPAINT_NS)
    source_depth_map = cv2.resize(source_depth_map, (source_w, source_h), interpolation=cv2.INTER_NEAREST)
    subsampled_mask = source_component_mask[::subsample_step, ::subsample_step]
    source_pixels_y_sub, source_pixels_x_sub = np.where(subsampled_mask > 0)
    source_pixels_y = source_pixels_y_sub * subsample_step
    source_pixels_x = source_pixels_x_sub * subsample_step
    source_pixels = np.vstack((source_pixels_x, source_pixels_y)).T
    source_image_model, source_camera_model = images[source_id], cameras[images[source_id].camera_id]
    point_cloud_3d = []
    for u, v in source_pixels:
        depth, _ = get_depth_from_map(u, v, source_depth_map)
        if depth:
            p_3d = back_project_point(u, v, depth, source_camera_model, source_image_model)
            if p_3d is not None: point_cloud_3d.append(p_3d)
    if not point_cloud_3d: return None
    point_cloud_3d = np.array(point_cloud_3d)
    shape_type = classify_component_shape(component_name)
    points_to_project = None
    if shape_type == 'planar':
        points_to_project = fit_plane_ransac(point_cloud_3d)
    else:
        if len(point_cloud_3d) < 4: return None
        tri = Delaunay(point_cloud_3d)
        points_to_project = point_cloud_3d[np.unique(tri.simplices)]
    if points_to_project is None or len(points_to_project) == 0: return None
    target_depth_map_raw = cv2.imread(image_id_to_depth_path[target_id], cv2.IMREAD_UNCHANGED)
    if target_depth_map_raw is None: return None
    target_depth_map = cv2.resize(target_depth_map_raw, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    visibility_checked_mask = np.zeros((target_h, target_w), dtype=np.uint8)
    target_image_model, target_camera_model = images[target_id], cameras[images[target_id].camera_id]
    R_target, t_target = qvec2rotmat(target_image_model.qvec), target_image_model.tvec
    for p_3d in points_to_project:
        p_cam_target = R_target @ p_3d + t_target
        z_projected = p_cam_target[2]
        if z_projected > 0:
            p_2d_target = project_point(p_3d, target_camera_model, target_image_model)
            if p_2d_target:
                u_t, v_t = int(round(p_2d_target[0])), int(round(p_2d_target[1]))
                if 0 <= v_t < target_h and 0 <= u_t < target_w:
                    z_ground_truth, _ = get_depth_from_map(u_t, v_t, target_depth_map)
                    if z_ground_truth and z_projected <= (z_ground_truth + depth_tolerance):
                        visibility_checked_mask[v_t, u_t] = 255
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    closed_mask = cv2.morphologyEx(visibility_checked_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    target_vehicle_mask = cv2.imread(image_id_to_mask_path.get(target_id, ''), cv2.IMREAD_GRAYSCALE)
    if target_vehicle_mask is not None:
        if target_vehicle_mask.shape != (target_h, target_w):
            target_vehicle_mask = cv2.resize(target_vehicle_mask, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
        return cv2.bitwise_and(closed_mask, target_vehicle_mask)
    else:
        return closed_mask

4 Manually selected degress source images, can be change with better one.
Also the stored results path can be change to your own path

In [ ]:
# ==========================================================
# =================== CONTROL PANEL ======================
# ==========================================================
main_output_dir = "Dataset2_Final_Manual_Analysis"
results_file_path = os.path.join(main_output_dir, "final_manual_results.json")
report_file_path = os.path.join(main_output_dir, "final_manual_report.txt")

# --- Manually define the 4 source images ---
manual_source_image_names = ["frame_00066.jpg", "frame_00191.jpg", "frame_00261.jpg", "frame_00770.jpg"]

# Create a mapping from image name to image ID for easy lookup
# This assumes the `images` dictionary is loaded and available at this point.
name_to_id_map = {img.name: img_id for img_id, img in images.items()}
manual_source_ids = [name_to_id_map[name] for name in manual_source_image_names if name in name_to_id_map]

# Check if all manual images were found
if len(manual_source_ids) != 4:
    print("Warning: Could not find all 4 manually specified source images in the dataset!")
    print(f"Found IDs: {manual_source_ids}")

# Analyze all available components from your dataset
# You can change this to your specific list if needed, e.g., ["bonnet", "bumper_f/cover", ...]
components_to_analyze = sorted(list(component_to_views_map.keys()))
# ==========================================================

# Create a 2x2 grid for the plots. `figsize` can be adjusted as needed.
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Visualization of the Four Manual Source Images", fontsize=20)

# Flatten the 2x2 `axes` array to make it easy to loop through (axes[0], axes[1], ...)
axes = axes.flatten()

# Loop through each filename and its corresponding subplot axis
for i, filename in enumerate(manual_source_image_names):
    # Construct the full path to the image
    image_path = os.path.join(rgb_images_path, filename)
    
    # Check if the file actually exists before trying to read it
    if os.path.exists(image_path):
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        # Convert from BGR (OpenCV's default) to RGB (Matplotlib's format)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Display the image on the corresponding subplot
        axes[i].imshow(image_rgb)
        axes[i].set_title(filename, fontsize=14)
    else:
        # If the image is not found, display a message on the plot
        axes[i].text(0.5, 0.5, f"Image not found:\n{filename}", 
                     ha='center', va='center', color='red')
        axes[i].set_title(filename, fontsize=14)

    # Turn off the axis ticks and labels for a cleaner look
    axes[i].axis('off')

# Adjust layout to prevent titles from overlapping and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust rect to make space for the suptitle
plt.show()

# Create a color map to assign a unique color to each component
num_components = len(components_to_analyze)
# Using 'gist_rainbow' colormap for a wide variety of distinct colors
colors = plt.cm.get_cmap('gist_rainbow', num_components)

# Create a 2x2 grid for the plots
fig, axes = plt.subplots(2, 2, figsize=(24, 18))
fig.suptitle("Ground Truth Masks on Source Images", fontsize=24)
axes = axes.flatten()

# Loop through each of the four source images
for i, filename in enumerate(manual_source_image_names):
    ax = axes[i]
    image_path = os.path.join(rgb_images_path, filename)
    
    if not os.path.exists(image_path):
        ax.text(0.5, 0.5, f"Image not found:\n{filename}", ha='center', va='center', color='red')
        ax.set_title(filename)
        ax.axis('off')
        continue

    # Read the image and create a copy to draw on
    image_rgb = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    overlay = image_rgb.copy()
    h, w, _ = image_rgb.shape
    
    legend_patches = []

    # Loop through all possible components to find their masks on this image
    for j, component_name in enumerate(components_to_analyze):
        mask, _ = get_component_mask_robust(filename, component_name, image_name_to_anns, cat_id_to_name, h, w)
        
        if mask is not None:
            # Get the color for this component (RGBA format)
            color = colors(j)
            # Convert to RGB for blending (and scale from 0-1 to 0-255)
            blend_color = np.array(color[:3]) * 255
            
            # Create the colored overlay for the mask
            roi = overlay[mask > 0]
            blended_roi = (roi * 0.5 + blend_color * 0.5).astype(np.uint8)
            overlay[mask > 0] = blended_roi
            
            # Add this component to the legend
            legend_patches.append(mpatches.Patch(color=color, label=component_name))

    # Display the final image with all masks overlaid
    ax.imshow(overlay)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')
    # Add the legend to the plot
    if legend_patches:
        ax.legend(handles=legend_patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 0.9, 0.95]) # Adjust rect for suptitle and legend
plt.show()

In [ ]:
# --- Load or initialize results ---
if os.path.exists(results_file_path):
    with open(results_file_path, 'r') as f:
        all_results = json.load(f)
else:
    all_results = {}

In [ ]:
# --- 1. Main Analysis Function (Modified to use manual sources) ---

def run_component_miou_analysis(component_name, existing_results):
    """
    Runs an exhaustive analysis for a component using the 4 manually selected source images.
    """
    print(f"\n{'='*25}\n--- Preparing Analysis for '{component_name}' ---\n{'='*25}")
    
    start_time_component = time.time()
    
    source_ids = manual_source_ids
    print(f"Using manually selected source images with IDs: {source_ids}")


    valid_targets = [tid for tid in valid_image_ids if tid not in source_ids and get_component_mask(images[tid].name, component_name, image_name_to_anns, cat_id_to_name)[0] is not None]
    if not valid_targets:
        print(f"No valid target images with ground truth found for '{component_name}'. Skipping.")
        return None
    
    print(f"Found {len(valid_targets)} valid targets to test against.")
    
    component_results = existing_results.copy() # Use existing results to resume
    
    # Create a list of targets that still need to be analyzed
    targets_to_run = [tid for tid in valid_targets if images[tid].name not in component_results]

    # Use tqdm for a progress bar over the selected targets
    for target_id in tqdm(targets_to_run, desc=f"Analyzing {component_name}"):
        target_name = images[target_id].name
        target_gt_mask, _ = get_component_mask(target_name, component_name, image_name_to_anns, cat_id_to_name)
        
        projected_masks = []
        for sid in source_ids:
            source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
            if source_mask is not None:
                transferred_mask = transfer_mask_v5(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
                if transferred_mask is not None:
                    projected_masks.append(transferred_mask)
        
        if not projected_masks:
            continue
            
        target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
        union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
        for mask in projected_masks:
            union_mask = cv2.bitwise_or(union_mask, mask)
        
        iou_score = calculate_iou(union_mask, target_gt_mask)
        
        # Save progress for this specific target
        component_results[target_name] = {"iou": iou_score}
        all_results[component_name] = component_results
        with open(results_file_path, 'w') as f:
            json.dump(all_results, f, indent=4)

    end_time_component = time.time()
    total_time = end_time_component - start_time_component
    
    # Add summary stats to the results for this component
    all_iou_scores = [res['iou'] for res in component_results.values()]
    component_results["individual_scores"] = all_iou_scores
    component_results["avg_iou"] = np.mean(all_iou_scores)
    component_results["total_time"] = total_time
    component_results["combinations_tested"] = len(all_iou_scores)

    return component_results


# --- Main Execution and Reporting Block ---

os.makedirs(main_output_dir, exist_ok=True)
if os.path.exists(results_file_path):
    print(f"Loading existing results from '{results_file_path}'...")
    with open(results_file_path, 'r') as f:
        all_results = json.load(f)
else:
    print("No existing results file found. Starting a new analysis.")
    all_results = {}

total_analysis_time = 0

for component_name in components_to_analyze:
    # --- Stage 1: Run the Analysis for the Component ---
    if component_name not in all_results or "avg_iou" not in all_results[component_name]:
        updated_results = run_component_miou_analysis(component_name, all_results.get(component_name, {}))
        if updated_results:
            all_results[component_name] = updated_results
            total_analysis_time += updated_results.get("total_time", 0)
            with open(results_file_path, 'w') as f:
                json.dump(all_results, f, indent=4)
    else:
        print(f"\nAnalysis for '{component_name}' already complete. Skipping computation.")

    # --- Stage 2: Generate and Save Visualization for the Component (Modified for Robustness) ---
    print(f"\n--- Generating visualization for '{component_name}' ---")

    # Define the output directory for this component's visualization
    component_output_dir = os.path.join(main_output_dir, component_name.replace('/', '_'))
    
    # Check if the directory exists and if there's already a PNG file in it
    # Note: You need to have 'import glob' at the top of your script for this to work.
    if os.path.isdir(component_output_dir) and glob.glob(os.path.join(component_output_dir, '*.png')):
        print(f"  - Visualization already exists for '{component_name}'. Skipping generation.")
        continue

    if component_name in all_results and all_results[component_name]:
        # Filter for actual target results, not summary keys
        successful_targets = [t for t in all_results[component_name].keys() if isinstance(all_results[component_name][t], dict) and 'iou' in all_results[component_name][t]]
        
        if not successful_targets:
            print("No successful targets to visualize for this component.")
            continue

        target_name_to_viz = random.choice(successful_targets)
        target_id_to_viz = next((img_id for img_id, img in images.items() if img.name == target_name_to_viz), None)
        
        if target_id_to_viz:
            print(f"  - Visualizing projection for target: {target_name_to_viz}")
            source_ids_viz = manual_source_ids
            
            projected_masks_viz = []
            source_masks_viz = []
            source_rgbs_viz = []

            # Loop through all source IDs and handle missing data
            for sid in source_ids_viz:
                source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB)
                source_h, source_w, _ = source_rgb.shape
                source_rgbs_viz.append(source_rgb) # Always add the source image
                
                source_mask, _ = get_component_mask_robust(images[sid].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
                source_masks_viz.append(source_mask) # Add the mask (or None if not found)
                
                # Only attempt to project if we found a source mask
                if source_mask is not None:
                    transferred_mask = transfer_mask_v5(sid, target_id_to_viz, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
                    projected_masks_viz.append(transferred_mask) # Add the result (or None if projection fails)
                else:
                    projected_masks_viz.append(None) # Add None if there was no mask to project
            
            # --- Create the detailed plot, now robust to missing masks ---
            num_sources = len(source_ids_viz)
            fig, axes = plt.subplots(3, num_sources, figsize=(10 * num_sources, 30))
            fig.suptitle(f"Detailed Visualization for '{component_name}' on {target_name_to_viz}", fontsize=30, y=0.95)
            target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id_to_viz]), cv2.COLOR_BGR2RGB)

            # Row 1: Source Masks
            for j in range(num_sources):
                ax = axes[0, j]
                ax.imshow(source_rgbs_viz[j])
                if source_masks_viz[j] is not None:
                    ax.imshow(np.ma.masked_where(source_masks_viz[j] == 0, source_masks_viz[j]), cmap='cool', alpha=0.6)
                    ax.set_title(f"Source {chr(65+j)}")
                else:
                    ax.set_title(f"Source {chr(65+j)}\n(Mask Not Found)")
                ax.axis('off')

            # Row 2: Individual Projected Masks
            for j in range(num_sources):
                ax = axes[1, j]
                ax.imshow(target_rgb)
                if projected_masks_viz[j] is not None:
                    ax.imshow(np.ma.masked_where(projected_masks_viz[j] == 0, projected_masks_viz[j]), cmap='plasma', alpha=0.6)
                    ax.set_title(f"Projection from Source {chr(65+j)}")
                else:
                    ax.set_title(f"Projection from Source {chr(65+j)}\n(No Source or Failed Projection)")
                ax.axis('off')

            # Row 3: Final Combined Results
            valid_projected_masks = [m for m in projected_masks_viz if m is not None]
            target_h, target_w, _ = target_rgb.shape
            union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
            if valid_projected_masks:
                for mask in valid_projected_masks:
                    union_mask = cv2.bitwise_or(union_mask, mask)
            
            target_gt_mask, _ = get_component_mask_robust(target_name_to_viz, component_name, image_name_to_anns, cat_id_to_name, target_h, target_w)
            iou_score = calculate_iou(union_mask, target_gt_mask) if target_gt_mask is not None else -1.0 # Handle missing GT
            
            # Plot Final Union
            axes[2, 0].imshow(target_rgb)
            axes[2, 0].imshow(np.ma.masked_where(union_mask == 0, union_mask), cmap='Greens', alpha=0.7)
            axes[2, 0].set_title(f"Final Union (IoU: {iou_score:.4f})")
            axes[2, 0].axis('off')
            
            # Plot Ground Truth
            axes[2, 1].imshow(target_rgb)
            if target_gt_mask is not None:
                axes[2, 1].imshow(np.ma.masked_where(target_gt_mask == 0, target_gt_mask), cmap='YlOrRd', alpha=0.7)
            axes[2, 1].set_title("Ground Truth")
            axes[2, 1].axis('off')
            
            # Turn off remaining axes in the last row
            for j in range(2, num_sources):
                axes[2, j].axis('off')

            component_output_dir = os.path.join(main_output_dir, component_name.replace('/', '_'))
            os.makedirs(component_output_dir, exist_ok=True)
            save_path = os.path.join(component_output_dir, f"visualization_{target_name_to_viz}.png")
            
            plt.tight_layout(rect=[0, 0, 1, 0.93])
            plt.savefig(save_path)
            plt.close(fig)
            print(f"  - Visualization saved to '{save_path}'")

# --- 4. Final Consolidated Report ---
print("\n\n" + "="*50)
print("      FINAL MEAN IoU (mIoU) REPORT")
print("="*50)
report_lines = []
overall_iou_scores = []

# Reload the most up-to-date results before printing the report
with open(results_file_path, 'r') as f:
    final_results_to_report = json.load(f)

for component, data in sorted(final_results_to_report.items()):
    if "avg_iou" in data:
        report_lines.append(f"\n--- Component: '{component}' ---")
        report_lines.append(f"  - Combinations Tested: {data['combinations_tested']}")
        report_lines.append(f"  - Total Time for Component: {data['total_time']:.2f} seconds")
        report_lines.append(f"  - Individual IoU Scores: {', '.join([f'{s:.4f}' for s in data['individual_scores']])}")
        report_lines.append(f"  >>> Mean IoU (mIoU): {data['avg_iou']:.4f}")
        overall_iou_scores.extend(data['individual_scores'])

if overall_iou_scores:
    overall_miou = np.mean(overall_iou_scores)
    report_lines.append("\n\n" + "="*50)
    report_lines.append("      OVERALL SUMMARY")
    report_lines.append("="*50)
    report_lines.append(f"\n- Total Number of Components Analyzed: {len(final_results_to_report)}")
    report_lines.append(f"- Total Time for All New Analyses: {total_analysis_time:.2f} seconds")
    report_lines.append(f">>> FINAL Overall mIoU across all components: {overall_miou:.4f} <<<")

final_report_string = "\n".join(report_lines)
with open(report_file_path, 'w') as f_out:
    f_out.write(final_report_string)

print(final_report_string)
print(f"\n\nAnalysis complete. Visualizations saved in '{main_output_dir}'. Full report saved to '{report_file_path}'.")

In [ ]:
# --- 1. Define the 3 Fixed Examples for Comparison ---
try:
    fixed_examples = [
        {
            "component": "bonnet",
            "source_id": component_to_views_map["bonnet"][0][1], # Best view
            "target_id": component_to_views_map["bonnet"][5][1]  # A different, good view
        },
        {
            "component": "windshield_f",
            "source_id": component_to_views_map["windshield_f"][0][1], # Best view
            "target_id": component_to_views_map["windshield_f"][3][1]   # A different, good view
        },
        {
            "component": "bumper_f/cover",
            "source_id": component_to_views_map["bumper_f/cover"][0][1], # Best view
            "target_id": component_to_views_map["bumper_f/cover"][7][1] # A different, good view
        }
    ]
except (KeyError, IndexError):
    print("Warning: Could not create fixed examples. Falling back to a single random example.")
    # Fallback to a single random example if the fixed ones aren't available
    component_name = random.choice(["bonnet", "bumper_f/cover", "windshield_f"])
    source_id = component_to_views_map.get(component_name, [])[0][1]
    target_id = random.choice([i for i in valid_image_ids if i != source_id and get_component_mask_robust(images[i].name, component_name, image_name_to_anns, cat_id_to_name, images[i].height, images[i].width)[0] is not None])
    fixed_examples = [{"component": component_name, "source_id": source_id, "target_id": target_id}]


# --- 2. Main Execution Block (Modified) ---
for i, example in enumerate(fixed_examples):
    component_name = example["component"]
    source_id = example["source_id"]
    target_id = example["target_id"]
    
    print(f"\n{'='*25}\n--- Running Comparison Example {i+1} for '{component_name}' ---\n{'='*25}")
    print(f"  - Source: {images[source_id].name}")
    print(f"  - Target: {images[target_id].name}")

    # --- Get Source and Target Masks ---
    source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
    source_h, source_w, _ = source_rgb.shape
    source_mask, _ = get_component_mask_robust(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
    
    target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
    target_h, target_w, _ = target_rgb.shape
    target_gt_mask, _ = get_component_mask_robust(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name, target_h, target_w)

    if source_mask is None or target_gt_mask is None:
        print("Could not get a valid source or target mask for this example. Skipping.")
        continue

    # --- Run All Transfer Methods, Calculate IoU, and Time ---
    masks = {}
    ious = {}
    times = {}
    
    print("\n--- Running Methods ---")
    
    # V1
    start_time = time.time(); masks['v1'] = transfer_mask_v1(source_id, target_id, source_mask, images, cameras, image_id_to_rgb_path, image_id_to_depth_path); times['v1'] = time.time() - start_time
    ious['v1'] = calculate_iou(masks['v1'], target_gt_mask)
    
    # V2
    start_time = time.time(); masks['v2'] = transfer_mask_v2(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path); times['v2'] = time.time() - start_time
    ious['v2'] = calculate_iou(masks['v2'], target_gt_mask)
    
    # V3
    start_time = time.time(); masks['v3'] = transfer_mask_v3(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v3'] = time.time() - start_time
    ious['v3'] = calculate_iou(masks['v3'], target_gt_mask)
    
    # --- NEW: V3_FAST ADDED HERE ---
    start_time = time.time(); masks['v3_fast'] = transfer_mask_v3_fast(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v3_fast'] = time.time() - start_time
    ious['v3_fast'] = calculate_iou(masks['v3_fast'], target_gt_mask)
    
    # V4
    start_time = time.time(); masks['v4'] = transfer_mask_v4(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v4'] = time.time() - start_time
    ious['v4'] = calculate_iou(masks['v4'], target_gt_mask)
    
    # V5
    start_time = time.time(); masks['v5'] = transfer_mask_v5(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v5'] = time.time() - start_time
    ious['v5'] = calculate_iou(masks['v5'], target_gt_mask)

    # --- UPDATED LIST OF VERSIONS ---
    versions_to_run = ['v1', 'v2', 'v3', 'v3_fast', 'v4', 'v5']

    print("\n--- Quantitative Results ---")
    for version in versions_to_run:
        print(f"  - {version.upper():<8}: Time = {times[version]:.2f}s | IoU = {ious[version]:.4f}")

    # --- Separate Visualization for Each Method ---
    for version in versions_to_run:
        fig, axes = plt.subplots(1, 3, figsize=(30, 10))
        fig.suptitle(f"Method {version.upper()} Comparison for '{component_name}'", fontsize=24)

        # Panel 1: Source
        axes[0].imshow(source_rgb); axes[0].imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.6)
        axes[0].set_title("1. Source Mask", fontsize=18); axes[0].axis('off')

        # Panel 2: Transferred Result
        axes[1].imshow(target_rgb)
        if masks[version] is not None:
            overlay_res = target_rgb.copy(); roi_res = overlay_res[masks[version] > 0]; blended_res = (roi_res * 0.5 + np.array([0, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8); overlay_res[masks[version] > 0] = blended_res; axes[1].imshow(overlay_res)
        axes[1].set_title(f"2. {version.upper()} Result\n(Time: {times[version]:.2f}s | IoU: {ious[version]:.4f})", fontsize=18)
        axes[1].axis('off')

        # Panel 3: Ground Truth
        axes[2].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt * 0.5 + np.array([255, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes[2].imshow(overlay_gt)
        axes[2].set_title("3. Ground Truth Mask", fontsize=18); axes[2].axis('off')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

print("\n\nAll comparison examples are complete.")